In [1]:
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
headers = []
csv_array = []

with open('ycd.csv', newline='') as ycd:
    reader = csv.reader(ycd, delimiter=',')
    line = 0
    for row in reader:
        if line == 0:
            data_rep = row
            line += 1
        else:
            csv_array.append(row)

In [71]:
pickup_time = 1
dropoff_time = 2
passenger_number = 3
pickup_location = 7
dropoff_location = 8
fare_amount = 10
tip_amount = 13
dropoff_rel_data = [pickup_time, pickup_location, dropoff_time]
tips_rel_data = [pickup_time, pickup_location, dropoff_location, passenger_number, tip_amount]

dropoff_data = []
tips_data = []
for row in csv_array:
    entry = []
    flag = True
    for item in dropoff_rel_data:
        if item == pickup_time or item == dropoff_time:
            entry.append(row[item].split()[1])
        elif item == pickup_location:
            if int(row[item]) > 263:
                flag = False
                break
            entry.append(int(row[item]))
            entry.append(0)
        elif row[item] != '':
            entry.append(float(row[item]))
        else:
            flag = False
            break
    if flag:
        dropoff_data.append(entry)
    entry = []
    flag = True
    for item in tips_rel_data:
        if item == pickup_time:
            entry.append(row[item].split()[1])
        elif item == pickup_location or item == dropoff_location:
            if int(row[item]) > 263:
                flag = False
                continue
            entry.append(int(row[item]))
            entry.append(0)
        elif row[item] != '':
            entry.append(float(row[item]))
        else:
            flag = False
            continue
    if flag:
        tips_data.append(entry)

In [72]:
coordinates = []
with open('coordinates.csv', newline='') as crds:
    reader = csv.reader(crds, delimiter=',')
    line = 0
    for row in reader:
        if line == 0:
            line += 1
        else:
            coordinates.append([float(row[1]), float(row[2])])
            line += 1

In [73]:
for entry in dropoff_data:
    # change first entry to time of day in seconds
    time = entry[0].split(':')
    time_secs = float(time[0]) * 3600 + float(time[1]) * 60 + float(time[2])
    entry[0] = time_secs
    # change location code to xy location
    entry[1] = coordinates[int(entry[1]) - 1][0]
    entry[2] = coordinates[int(entry[1]) - 1][1]
    # change last entry to minutes elapsed during trip
    time = entry[3].split(':')
    time_secs = float(time[0]) * 3600 + float(time[1]) * 60 + float(time[2])
    time_secs -= entry[0]
    if time_secs < 0:
        time_secs += 24 * 3600
    entry[3] = time_secs / 60
    # change first entry to represent time in hours
    entry[0] /= 3600

for entry in tips_data:
    # change date time to time of day in minutes
    time = entry[0].split(':')
    time_hrs = float(time[0]) + float(time[1]) / 60 + float(time[2]) / 3600
    entry[0] = time_mins
    # change location code to xy location
    entry[1] = coordinates[int(entry[1]) - 1][0]
    entry[2] = coordinates[int(entry[1]) - 1][1]
    entry[3] = coordinates[int(entry[2]) - 1][0]
    entry[4] = coordinates[int(entry[2]) - 1][1]

In [74]:
dropoff_titles = ["Pickup Time (in Hours)", "Pickup X Coordinate", "Pickup Y Coordinate", "Trip Duration (in Minutes)"]
tips_titles = ["Pickup Time (in Hours)", "Pickup X Coordinate", "Pickup Y Coordinate", "Dropoff X Coordinate", "Dropoff Y Coordinate", "Passenger Number", "Tip Amount"]

with open('prediction.csv', 'w', newline='') as pred:
    writer = csv.writer(pred, delimiter=',')
    writer.writerow(dropoff_titles)
    for row in dropoff_data:
        writer.writerow(row)

with open('regression.csv', 'w', newline='') as regr:
    writer = csv.writer(regr, delimiter=',')
    writer.writerow(tips_titles)
    for row in tips_data:
        writer.writerow(row)